In [24]:
import pandas as pd

orders=pd.read_csv('../staging_1/orders/orders.csv')
orders.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extraction_time,source
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-04-29 12:47:30.938683,postgres
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-04-29 12:47:30.938683,postgres
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-04-29 12:47:30.938683,postgres
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-04-29 12:47:30.938683,postgres
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-04-29 12:47:30.938683,postgres


In [25]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1432 entries, 0 to 1431
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   order_id         1432 non-null   int64 
 1   customer_id      1432 non-null   int64 
 2   order_status     1432 non-null   int64 
 3   order_date       1432 non-null   object
 4   required_date    1432 non-null   object
 5   shipped_date     1432 non-null   object
 6   store_id         1432 non-null   int64 
 7   staff_id         1432 non-null   int64 
 8   extraction_time  1432 non-null   object
 9   source           1432 non-null   object
dtypes: int64(5), object(5)
memory usage: 112.0+ KB


In [26]:
#conversion to datetime so we could compare 

orders['required_date'] = pd.to_datetime(orders['required_date'] ,format='%Y-%m-%d' )
orders['shipped_date'] = pd.to_datetime(orders['shipped_date'] ,format='%Y-%m-%d' )

In [38]:
#add late_delivery column
orders['late_delivery'] = orders['shipped_date'] > orders['required_date']

In [47]:
orders.head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extraction_time,source,late_delivery,latency_days
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-04-29 12:47:30.938683,postgres,False,None
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-04-29 12:47:30.938683,postgres,False,None
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-04-29 12:47:30.938683,postgres,False,None
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-04-29 12:47:30.938683,postgres,True,1
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-04-29 12:47:30.938683,postgres,False,None


In [48]:

# initialize a none value for the whole column
orders['latency_days'] = None 

for index, row in orders.iterrows():
    if row['late_delivery']:  # Check if the delivery is late
        orders.at[index, 'latency_days'] = (row['shipped_date'] - row['required_date']).days


In [50]:
#add new column “weekend” 
orders['weekend'] = (orders['shipped_date'].dt.dayofweek.isin([6, 0])).astype(bool)
merged_df.to_csv(('../staging_2/order_items/order_items.csv')).head()

,order_id,customer_id,order_status,order_date,required_date,shipped_date,store_id,staff_id,extraction_time,source,late_delivery,latency_days,weekend
0,1,259,4,2016-01-01,2016-01-03,2016-01-03,1,2,2024-04-29 12:47:30.938683,postgres,False,None,True
1,2,1212,4,2016-01-01,2016-01-04,2016-01-03,2,6,2024-04-29 12:47:30.938683,postgres,False,None,True
2,3,523,4,2016-01-02,2016-01-05,2016-01-03,2,7,2024-04-29 12:47:30.938683,postgres,False,None,True
3,4,175,4,2016-01-03,2016-01-04,2016-01-05,1,3,2024-04-29 12:47:30.938683,postgres,True,1,False
4,5,1324,4,2016-01-03,2016-01-06,2016-01-06,2,6,2024-04-29 12:47:30.938683,postgres,False,None,False


In [52]:
orders.to_csv('../staging_2/orders/orders.csv' , index = False)